In [12]:
from bokeh.plotting import figure, save
from bokeh.io import output_file, show
from bokeh.models import ColumnDataSource, HoverTool, LogColorMapper, GMapPlot, GMapOptions, ColumnDataSource, Circle, DataRange1d, PanTool, WheelZoomTool, BoxSelectTool
from bokeh.io import show, output_notebook
from bokeh.models import GeoJSONDataSource, LinearColorMapper

import geopandas as gpd
from bokeh.palettes import Viridis6 as palette
import geopandas as gpd
import pysal as ps
import pandas as pd
import numpy as np
import math

In [36]:
# Parastoo's pwd
# grid_fp = r"C:/Users/jabbari/Desktop/uwseds-group-transit-and-social-science/Data/tract2010/tract2010.shp"
# network_fp = r"C:/Users/jabbari/Desktop/uwseds-group-transit-and-social-science/Transit Routes/Transit_Routes_for_King_County_Metro__transitroute_line.shp"

# Liam's pwd
grid_fp = r"/home/liam/uwseds/Project/uwseds-group-transit-and-social-science/Data/tract2010"
network_fp = r"/home/liam/uwseds/Project/uwseds-group-transit-and-social-science/TransitRoutes"


grid = gpd.read_file(grid_fp)
network = gpd.read_file(network_fp)
grid = grid.loc[grid['COUNTYFP10'] == '033']

network_source = GeoJSONDataSource(geojson=network.to_json())


In [47]:
network

,OBJECTID,CHANGE_NUM,MINOR_CHAN,CURRENT_NE,IN_SERVICE,ROUTE_ID,LOCAL_EXPR,ROUTE_NUM,SHAPE_Leng,geometry
0,1,137,3,SEPT17,Y,,,,5.187577e+06,(LINESTRING (-121.9822100015218 47.19997397393...
1,2,137,3,SEPT17,Y,100001,L,1,4.249743e+04,(LINESTRING (-122.3177498661453 47.59918793133...
2,3,137,3,SEPT17,Y,100002,L,10,2.934673e+04,(LINESTRING (-122.3398126258551 47.61027837008...
3,4,137,3,SEPT17,Y,100003,L,101,1.347552e+05,(LINESTRING (-122.2097200019206 47.47252798095...
4,5,137,3,SEPT17,Y,100004,L,105,3.623683e+04,(LINESTRING (-122.2029776399569 47.48214733583...
5,6,137,3,SEPT17,Y,100005,L,106,1.008968e+05,(LINESTRING (-122.217189050582 47.481347612252...
6,7,137,3,SEPT17,Y,100006,L,107,1.117883e+05,(LINESTRING (-122.2091527398079 47.47970911502...
7,8,137,3,SEPT17,Y,100009,L,11,4.106447e+04,(LINESTRING (-122.3377930449671 47.60974983276...
8,9,137,3,SEPT17,Y,100011,L,111,2.179385e+05,"(LINESTRING (-122.1151955730682 47.4735675319,..."
9,10,137,3,SEPT17,Y,100012,L,113,9.408574e+04,(LINESTRING (-122.3657841475034 47.50099626558...


In [17]:
# Parastoo's pwd
#income_df = pd.read_csv('C:/Users/jabbari/Desktop/uwseds-group-transit-and-social-science/Docs/income_tract.csv')

# Liam's pwd
#income_df = pd.read_csv(r"/home/liam/uwseds/Project/uwseds-group-transit-and-social-science/Data/tract2010")

In [18]:
grid['geometry'] = grid['geometry'].to_crs(crs='+init=epsg:4326')

In [19]:
def getXYCoords(geometry, coord_type):
    """ Returns either x or y coordinates from  geometry coordinate sequence. Used with LineString and Polygon geometries."""
    if coord_type == 'x':
        return geometry.coords.xy[0]
    elif coord_type == 'y':
        return geometry.coords.xy[1]
    
def getLineCoords(geometry, coord_type):
    """ Returns Coordinates of Linestring object."""
    return getXYCoords(geometry, coord_type)

def getPolyCoords(geometry, coord_type):
    """ Returns Coordinates of Polygon using the Exterior of the Polygon."""
    ext = geometry.exterior
    return getXYCoords(ext, coord_type)

def getPointCoords(geometry, coord_type):
    """ Returns Coordinates of Point object."""
    if coord_type == 'x':
        return geometry.x
    elif coord_type == 'y':
        return geometry.y


def getCoords(row, geom_col, coord_type):
    """
    Returns coordinates ('x' or 'y') of a geometry (Point, LineString or Polygon) as a list (if geometry is LineString or Polygon). 
    Can handle also MultiGeometries.
    """
    # Get geometry
    geom = row[geom_col]
    
    # Check the geometry type
    gtype = geom.geom_type
    
    # "Normal" geometries
    # -------------------
    
    if gtype == "Point":
        return getPointCoords(geom, coord_type)
    elif gtype == "LineString":
        return list( getLineCoords(geom, coord_type) )
    elif gtype == "Polygon":
        return list( getPolyCoords(geom, coord_type) )


grid['x'] = grid.apply(getCoords, geom_col="geometry", coord_type="x", axis=1)
grid['y'] = grid.apply(getCoords, geom_col="geometry", coord_type="y", axis=1)


In [20]:
g_df = grid.drop('geometry', axis=1).copy()
gsource = ColumnDataSource(g_df)


In [22]:
#income_df = pd.read_csv('C:/Users/jabbari/Desktop/uwseds-group-transit-and-social-science/Data/income_tract.csv')

In [29]:
#income_df['htract'].head ()

In [30]:
#grid['NAME10'] = pd.to_numeric(grid['NAME10'])

In [33]:
#grid = grid.merge(income_df, left_on='NAME10', right_on='htract', how='inner')

In [34]:
# breaks = [x for x in range(1, 11, 2)]

# # Initialize the classifier and apply it
# classifier = ps.User_Defined.make(bins=breaks)
# pt_classif = grid[['hh_income_detailed']].apply(classifier)

# # Rename the classified column
# pt_classif.columns = ['income2']

# # Join it back to the grid layer
# grid = grid.join(pt_classif)

In [35]:
from bokeh.palettes import RdYlGn10 as palette
from bokeh.models import LogColorMapper

# Create the color mapper

color_mapper = LogColorMapper(palette=palette)

p = figure(title="PSRC Income")

# Plot grid
grid2 = p.patches('x', 'y', source=gsource,
         fill_color='white',
         fill_alpha=1.0, line_color="black", line_width=0.05, legend="income2")

#r = p.multi_line('x', 'y', source=nsource, color="red", line_width=2)

#p.legend.location = "top_right"
#p.legend.orientation = "vertical"

p.multi_line('xs', 'ys', source=network_source, color="red", line_width=0.5)

#ghover = HoverTool(renderers=[grid2])
#ghover.tooltips=[("Income Category", "@income2")]
              

#p.add_tools(ghover)

#outfp = r"C:/Users/jabbari/Desktop/uwseds-group-transit-and-social-science/Docs/map.html"
outfp = r"/home/liam/Desktop/map.html"
save(p, outfp)

show(p)

/home/liam/anaconda3/lib/python3.6/site-packages/bokeh/io.py:527: UserWarning: save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN
  warnings.warn("save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN")
/home/liam/anaconda3/lib/python3.6/site-packages/bokeh/io.py:537: UserWarning: save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'
  warnings.warn("save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'")


RuntimeError: Sub-model (id='edfdade3-1f91-4309-a670-214d1aa17833', ...) of the root model Figure(id='5056fb5b-9601-49d7-b864-e94ed98fedac', ...) is already owned by another document (Models must be owned by only a single document). This may indicate a usage error.